In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def bin_maker(data_img):
    binary_array=np.unpackbits(data_img, axis=-1)
    binary_array=binary_array.reshape(data_img.shape[0],data_img.shape[1],-1,2)
    flat_array=binary_array.flatten()
    return flat_array,len(flat_array)

In [3]:
def bin_pack(data_img):
    binary_array=data_img.reshape(64,64,4,2)
    # Reshape the grouped binary array to match the original shape
    reshaped_array = binary_array.reshape(binary_array.shape[0],binary_array.shape[1], -1)

    # Convert each grouped binary representation back to integers
    int_array = np.packbits(reshaped_array, axis=-1).reshape(binary_array.shape[0],binary_array.shape[1])

    return int_array


In [4]:
def hide_image(cover_path,secret_path,output_path):
    cover =cv2.imread(cover_path,cv2.IMREAD_GRAYSCALE)
    secret =cv2.imread(secret_path,cv2.IMREAD_GRAYSCALE)
    
    secret_rs=cv2.resize(secret,(64,64),cv2.INTER_AREA)
    
    bin_array,size=bin_maker(secret_rs)
    
    stego=np.copy(cover)
    rows,cols=cover.shape
    
    index=0
    for row in range(rows) :
        for col in range(cols):
            cover_pixel=cover[row,col]
            stego[row,col]={cover_pixel & 0xFE} | {bin_array[index]}
            index+=1
            if(index==size):
                break
        if(index==size):
            break
        
    cv2.imwrite(output_path,stego)
    print("image hidden succesfully")
            

In [6]:
def reveal_image(stego_path, output_path):
    stego = cv2.imread(stego_path, cv2.IMREAD_GRAYSCALE)
    xplc=cv2.resize(stego,(64,64),cv2.INTER_AREA)
    revealed = np.zeros_like(xplc)

    size=32768
    index = 0
    rows, cols = stego.shape

    for row in range(rows):
        for col in range(cols):
            stego_pixel = stego[row, col]
            # Extract the least significant bit as the secret pixel's value
            revealed_pixel = (stego_pixel & 0x01) << 7
            revealed[row, col] = revealed_pixel
            index+=1
            if(index==size):
                break
        if (index==size):
            break

    cv2.imwrite(output_path, revealed)
    print("Secret image revealed successfully!")